# Investigation of effect of active dendritic conductances on spike shapes

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import LFPy
import neuron
from neuron import h
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots
from brainsignals.neural_simulations import return_hay_cell

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(12345)

tstop = 150
dt = 2**-6

# Time window to extract spike from:
t0 = 118
t1 = 123

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell


def return_electrode_grid():
    xmin, xmax = [-50, 70]
    zmin, zmax = [-50, 130]

    dx = 20
    dz = 20
    x_grid, z_grid = np.mgrid[xmin:xmax+dx:dx, zmin:zmax+dz:dz]
    num_elecs = len(x_grid.flatten())
    elec_grid_params = dict(
                sigma = 0.3,      # extracellular conductivity
                x = x_grid.flatten(),
                y = np.zeros(num_elecs),
                z = z_grid.flatten(),
                method = 'linesource',
            )
    return elec_grid_params

## First we run original simulation with active conductances.  The somatic membrane potential from this simulation will be replayed into passive version

In [ ]:
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)

apic_idx = cell.get_closest_idx(x=15, y=0, z=300)

ns.point_axon_down(cell)
syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

apic_vm_active = cell.vmem[apic_idx, :].copy()

imem_orig = cell.imem.copy()
soma_t_orig = cell.tvec.copy()
soma_v_orig = cell.vmem[0, :].copy()

elec_grid_params = return_electrode_grid()

elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
M_elec = elec.get_transformation_matrix()
eaps_orig = M_elec @ cell.imem * 1000

cell.__del__()

## Then we replay the simulated somatic membrane potential into the soma of the passive version of the model

In [ ]:
cell = return_hay_cell(tstop=(soma_t_orig[-1] - 0.5 * dt), dt=dt, make_passive=False)

ns.point_axon_down(cell)
remove_list = ["Nap_Et2", "NaTa_t", "NaTs2_t", "SKv3_1",
               "SK_E2", "K_Tst", "K_Pst",
               "Im", "Ih", 
               "CaDynamics_E2", "Ca_LVAst", "Ca", "Ca_HVA"]
cell = ns.remove_active_mechanisms(remove_list, cell)
h.dt = dt

for sec in neuron.h.allsec():
    if "soma" in sec.name():
        vclamp = h.SEClamp_i(sec(0.5))
        vclamp.dur1 = 1e9
        vclamp.rs = 1e-9
        vmem_to_insert = h.Vector(soma_v_orig[1:])
        vmem_to_insert.play(vclamp._ref_amp1, h.dt)

cell.simulate(rec_imem=True, rec_vmem=True)

apic_vm_passive = cell.vmem[apic_idx, :].copy()

imem_repl = cell.imem.copy()
elec_grid_params = return_electrode_grid()
elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
M_elec = elec.get_transformation_matrix()
eaps_repl = M_elec @ cell.imem * 1000

## Now we compare the spikes

In [ ]:
xmin = np.min(elec_grid_params["x"])
xmax = np.max(elec_grid_params["x"])
zmin = np.min(elec_grid_params["z"])
zmax = np.max(elec_grid_params["z"])

eap_idxs = np.where((np.abs(elec_grid_params["z"] + 10) < 1e-9) &
                    (elec_grid_params["x"] > 0))[0]

eap_clrs = {idx: plt.cm.Reds_r(num / (len(eap_idxs)))
            for num, idx in enumerate(eap_idxs)}

fig = plt.figure(figsize=[6, 4.2])

ax_morph = fig.add_axes([0.02, 0.1, 0.45, 0.85], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin - 5, xmax + 10],
                        ylim=[zmin - 10, zmax + 5])

ax_diff = fig.add_axes([0.52, 0.1, 0.45, 0.85], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin - 5, xmax + 10],
                        ylim=[zmin - 10, zmax + 5])

zips = []
for x, z in cell.get_pt3d_polygons():
    zips.append(list(zip(x, z)))
polycol = PolyCollection(zips, edgecolors='none',
                         facecolors='0.8', zorder=-1, rasterized=False)
ax_morph.add_collection(polycol)
polycol = PolyCollection(zips, edgecolors='none',
                         facecolors='0.8', zorder=-1, rasterized=False)
ax_diff.add_collection(polycol)


dz = np.abs(np.diff(elec.z))[0]
num_elecs = len(elec.x)
eap_norm = dz * 0.9 / np.max(np.abs(eaps_orig))
eap_norm_diff = dz * 0.9 / np.max(np.abs(eaps_orig - eaps_repl))
t_norm_orig = soma_t_orig / soma_t_orig[-1] * dz * 0.7
t_norm_repl = cell.tvec / cell.tvec[-1] * dz * 0.7

for elec_idx in range(num_elecs):
    x, z = elec.x[elec_idx], elec.z[elec_idx]
    ax_morph.plot(x, z, '.', c='k', ms=3)
    eap_orig = eaps_orig[elec_idx] * eap_norm
    eap_repl = eaps_repl[elec_idx] * eap_norm
    eap_diff = (eaps_orig[elec_idx] - eaps_repl[elec_idx]) * eap_norm_diff
    l1, = ax_morph.plot(x + t_norm_orig, z + eap_orig, c='k', lw=1.5)
    l2, = ax_morph.plot(x + t_norm_repl, z + eap_repl, c='r', ls="-", lw=1)
    l3, = ax_diff.plot(x + t_norm_repl, z + eap_diff, c='b', ls="-", lw=1.5)


ax_morph.plot([20, 40], [15, 15], c='gray', lw=1)
ax_morph.text(30, 17, "20 µm", ha="center", c='gray')
ax_diff.plot([20, 40], [15, 15], c='gray', lw=1)
ax_diff.text(30, 17, "20 µm", ha="center", c='gray')


ax_morph.plot([82, 82], [10 - 300 * eap_norm, 10], c='k', lw=1,
              clip_on=False)
ax_morph.text(79, 0, "300 µV", ha="right",
              c='k', va="center")

ax_diff.plot([82, 82], [10 - 10 * eap_norm_diff, 10], c='k', lw=1,
              clip_on=False)
ax_diff.text(79, 0, "10 µV", ha="right",
              c='k', va="center")
fig.legend([l1, l2, l3], ["original active", "passive replay", "difference"],
          frameon=False, ncol=3, loc="lower right")


mark_subplots(fig.axes, ypos=1.01, xpos=0.)
fig.savefig("active_dendrites_compare_diff.pdf")

In [ ]:
fig2 = plt.figure()
ax1 = fig2.add_subplot(111)
ax1.plot(cell.tvec, apic_vm_active, 'k')
ax1.plot(cell.tvec, apic_vm_passive, 'r')
